# Mini-R1: 复现Deepseek R1的"顿悟时刻"——一个强化学习教程

Deepseek R1的发布震撼了整个行业。为什么？因为DeepSeek-R1是一个开源模型，在复杂推理任务中能与OpenAI的o1匹敌，其核心创新是使用组相对策略优化（Group Relative Policy Optimization, GRPO）和以强化学习（RL）为核心的多阶段训练方法。他们不仅发布了模型，还公开了相关研究论文。

论文中描述了使用纯强化学习训练模型时的"顿悟时刻"。在此阶段，DeepSeek-R1-Zero（DeepSeek-R1的初期版本）学会了通过重新评估初始策略来为问题分配更多思考时间，而无需任何人工反馈或数据指导。他们将此称为"顿悟时刻"：

> 这种行为不仅证明了模型推理能力的提升，更展示了强化学习如何带来意想不到的复杂结果。

这提醒我们，强化学习有潜力解锁人工智能系统的新智能层次，为未来更自主、自适应的模型铺平道路。本文将通过使用GRPO和《倒计时游戏》（Countdown Game）复现DeepSeek-R1的"小顿悟时刻"。我们将训练一个开源模型，利用强化学习使其自主掌握自我验证和搜索能力来解决《倒计时游戏》。该游戏的规则是：玩家使用一组随机抽取的数字和基本算术运算（+、-、×、÷）来达成或尽可能接近目标数字。

```
目标数字: 952
可用数字: 25, 50, 75, 100, 3, 6

(100 × (3 × 3)) + (50 + 6 / 3) = 952
```

本文包含可在Jupyter Notebook中运行的交互式代码，展示如何使用GRPO和Q-LoRa训练模型。虽然这种方法能帮助学习TRL和GRPO，但速度较慢且需要大量算力。此外，我还提供了[脚本](https://github.com/philschmid/deep-learning-pytorch-huggingface/blob/main/training/scripts/run_r1_grpo.py)和指南，支持在多GPU环境或SLURM集群上运行训练。

1. [配置开发环境](#1-配置开发环境)
2. [从《倒计时游戏》生成带有推理前缀的训练样本](#2-从倒计时游戏生成带有推理前缀的训练样本)
3. [使用GRPO训练模型](#3-使用grpo训练模型)
4. [使用Deepspeed和vLLM进行GRPO分布式训练示例](#4-使用deepspeed和vllm进行grpo分布式训练示例)
5. [结果与训练观察](#5-结果与训练观察)

_注：本文灵感源自[Jiayi Pan](https://x.com/jiayi_pirate/status/1882839370505621655)，他率先探索了这一思路并用小模型验证了可行性。_

在开始前，我们先了解[组相对策略优化（GRPO）](https://arxiv.org/abs/2402.03300)的工作原理。

## 组相对策略优化（GRPO）

GRPO是一种用于提升大语言模型（LLM）推理能力的强化学习算法，首次在[DeepSeekMath](https://arxiv.org/abs/2402.03300)论文中提出。GRPO改进了传统的近端策略优化（PPO），通过消除对价值函数模型的需求，转而从组分数中估计基线，从而降低内存和计算开销。Qwen团队也采用了GRPO，可结合规则/二元奖励或通用奖励模型提升模型的有用性。

1. **采样**：使用当前策略为每个提示生成多个输出。
2. **奖励评分**：使用奖励函数为每个生成结果评分（规则或结果导向）。
3. **优势计算**：以组内生成结果的平均奖励为基线，计算每个解决方案的相对优势（奖励在组内归一化）。
4. **策略优化**：通过最大化GRPO目标函数（包含计算优势和KL散度项）优化策略。

![grpo.png](../assets/grpo.png)

## 1. 配置开发环境

第一步是安装Hugging Face库、PyTorch、vLLM、trl、transformers和datasets。如果你不熟悉trl，它是一个基于transformers和datasets的库，可简化开源LLM的微调、RLHF和对齐过程。

In [ ]:
# 安装PyTorch及其他库（确保与GPU驱动版本匹配）
%pip install "torch==2.5.1" tensorboard "setuptools<71.0.0"  --index-url https://download.pytorch.org/whl/cu121

# 安装flash-attn
%pip install flash-attn 

# 安装Hugging Face库
%pip install  --upgrade \
  "transformers==4.48.1" \
  "datasets==3.1.0" \
  "accelerate==1.3.0" \
  "hf-transfer==0.1.9" \
  "deepspeed==0.15.4" \
  "trl==0.14.0"

# 安装vLLM
%pip install "vllm==0.7.0"

## 重要提示：如需运行交互式代码，还需安装以下库（可能与分布式训练库冲突）
# %pip install "peft==0.14.0" "bitsandbytes==0.45.0"

_注：安装后可能需要重启内核以应用更新。_

我们将使用[Hugging Face Hub](https://huggingface.co/models)作为远程模型版本控制服务。训练过程中，模型、日志和信息会自动推送至Hub。请先注册[Hugging Face账号](https://huggingface.co/join)，然后使用`huggingface_hub`的`login`工具登录并存储访问令牌。

In [ ]:
from huggingface_hub import login

login(token="", add_to_git_credential=True) # 在此填入你的令牌

## 2. 从《倒计时游戏》生成带有推理前缀的训练样本

我们将使用数据集[Jiayi-Pan/Countdown-Tasks-3to4](https://huggingface.co/datasets/Jiayi-Pan/Countdown-Tasks-3to4)，其中包含3到4个数字的样本及解法。

模型选用[Qwen/Qwen2.5-3B-Instruct](https://huggingface.co/Qwen/Qwen2.5-3B-Instruct)——一个30亿参数的指令调优模型。该模型已适配提示格式，便于展示"顿悟时刻"。当然，你也可以使用Qwen的基础版本或其他模型。[Jiayi Pan](https://x.com/jiayi_pirate/status/1882839487417561307)发现模型需具备一定规模（>15亿参数）才能学习推理过程。

In [ ]:
from transformers import AutoTokenizer
from datasets import load_dataset

# 从Hugging Face Hub加载数据集
dataset_id = "Jiayi-Pan/Countdown-Tasks-3to4"
dataset = load_dataset(dataset_id, split="train")
# 随机选取50k样本
dataset = dataset.shuffle(seed=42).select(range(50000))

# 加载分词器以格式化数据集为"r1"提示
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-3B-Instruct")

# 生成带有推理前缀的r1提示
def generate_r1_prompt(numbers, target):
    r1_prefix = [{
        "role": "system",
        "content": "你是一个乐于助人的助手。你首先在脑海中思考推理过程，然后为用户提供答案。"
      },
      { 
        "role": "user",
        "content": f"使用数字{numbers}创建一个等于{target}的方程。可使用基本算术运算（+、-、*、/），每个数字仅用一次。在<think></think>标签中展示思考过程，并在<answer></answer>标签中返回最终方程和答案。示例：<answer>(1 + 2) / 3 = 1</answer>。"
      },
      {
        "role": "assistant",
        "content": "让我一步步解决这个问题。\n<think>"
      }]
    return {"prompt": tokenizer.apply_chat_template(r1_prefix, tokenize=False, continue_final_message=True), "target": target}

# 将数据集转换为r1提示
dataset = dataset.map(lambda x: generate_r1_prompt(x["nums"], x["target"]))

# 划分训练集和测试集
train_test_split = dataset.train_test_split(test_size=0.1)

train_dataset = train_test_split["train"]
test_dataset = train_test_split["test"]

## 3. 使用GRPO训练模型

_注：本节演示如何使用TRL和GRPO的基础流程。如需运行交互式代码，需安装`bitsandbytes`和`peft`。本节主要用于教学目的。_

TRL通过专用[GRPOTrainer](https://huggingface.co/docs/trl/main/en/grpo_trainer)支持GRPO，用于根据偏好数据对齐LLM（如[DeepSeekMath论文](https://arxiv.org/abs/2402.03300)所述）。`GRPOTrainer`是`transformers`库中`Trainer`的子类，支持日志记录、检查点、分布式训练和参数高效微调（PEFT）等功能。

`GRPOTrainer`支持通用结果奖励模型（ORM）和自定义奖励函数。在Deepseek R1论文中，他们使用规则奖励模型验证生成解法的正确性。本例将采用类似方法，创建两个奖励函数：
1. **格式奖励**：检查生成格式是否正确`<think>...</think><answer>...</answer>`。
2. **准确性奖励**：从`<answer>`标签中提取方程，评估其是否匹配目标数字且所有数字仅用一次。

In [16]:
import re

def format_reward_func(completions, target, **kwargs):
    """
    格式检查：<think>...</think><answer>...</answer>
    参数：
        completions (list[str]): 生成输出
        target (list[str]): 预期答案
    返回：
        list[float]: 奖励分数
    """
    rewards = []
    for completion, gt in zip(completions, target):
        try:
            # 添加<think>前缀以匹配正则
            completion = "<think>" + completion
            regex = r"^<think>([^<]*(?:<(?!/?think>)[^<]*)*)<\/think>\n<answer>([\s\S]*?)<\/answer>$"

            match = re.search(regex, completion, re.DOTALL)
            rewards.append(1.0 if match and len(match.groups()) == 2 else 0.0)
        except Exception:
            rewards.append(0.0)
    return rewards

def equation_reward_func(completions, target, nums, **kwargs):
    """
    评估方程正确性：
    1. 方程数学正确性
    2. 所有数字仅使用一次
    参数：
        completions (list[str]): 生成输出
        target (list[str]): 目标数字
        nums (list[str]): 可用数字
    返回：
        list[float]: 奖励分数
    """
    rewards = []
    for completion, gt, numbers in zip(completions, target, nums):
        try:
            completion = "<think>" + completion
            match = re.search(r"<answer>(.*?)<\/answer>", completion)
            if not match:
                rewards.append(0.0)
                continue
            equation = match.group(1).strip()
            used_numbers = [int(n) for n in re.findall(r'\d+', equation)]
            # 检查数字使用情况
            if sorted(used_numbers) != sorted(numbers):
                rewards.append(0.0)
                continue
            # 允许的字符检查
            if not re.match(r'^[\d+\-*/().\s]+$', equation):
                rewards.append(0.0)
                continue
            # 方程求值
            result = eval(equation, {"__builtins__": None}, {})
            rewards.append(1.0 if abs(float(result) - float(gt)) < 1e-5 else 0.0)
        except Exception:
            rewards.append(0.0)
    return rewards

测试奖励函数（注意：示例中未包含初始`<think>`标签，我们已通过代码自动添加）：

In [17]:
correct_sample_1 = """需使用数字19, 36, 55, 7各一次，通过基本运算得到65。可能的解法是55 + 36 - 19 + 7... </think>
<answer> 55 + 36 - 7 - 19 </answer>"""

correct_sample_2 = """ ... </think>
<answer> 55 + 36 - 7 - 19 </answer>"""

wrong_format = """用户：使用数字[19, 36, 55, 7]创建等于65的方程。"""

wrong_format_2 = """尝试用95、78、6、88得到79：
95 + 88 = 183
183 - 104 = 79
<think> 183 - 104 = 79 </think><think> 183 - 104 = 79 </think><answer> 183 - 104 = 79 </answer>"""

wrong_result = """ ... </think>
<answer> 55 + 36 - 7 - 18 </answer>"""

test_rewards = format_reward_func(
    completions=[correct_sample_1, correct_sample_2, wrong_format, wrong_format_2, wrong_result],
    target=["65", "65", "65", "65", "65"],
    nums=[[19, 36, 55, 7]] * 5
)
assert test_rewards == [1.0, 1.0, 0.0, 0.0, 1.0], "格式奖励函数异常"

test_rewards = equation_reward_func(
    completions=[correct_sample_1, correct_sample_2, wrong_format, wrong_format_2, wrong_result],
    target=["65", "65", "65", "65", "65"],
    nums=[[19, 36, 55, 7]] * 5
)
assert test_rewards == [1.0, 1.0, 0.0, 0.0, 0.0], "方程奖励函数异常"

接下来定义训练参数并创建训练器：

In [ ]:
from trl import GRPOConfig, GRPOTrainer, get_peft_config, ModelConfig

# 模型配置
model_config = ModelConfig(
    model_name_or_path="Qwen/Qwen2.5-3B-Instruct",
    torch_dtype="bfloat16",
    attn_implementation="flash_attention_2",
    use_peft=True,
    load_in_4bit=True,
)

# 超参数配置
training_args = GRPOConfig(
    output_dir="qwen-r1-aha-moment",
    learning_rate=5e-7,
    lr_scheduler_type="cosine",
    logging_steps=10,
    max_steps=100,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=1,
    gradient_checkpointing=True,
    gradient_checkpointing_kwargs={"use_reentrant": False},
    bf16=True,
    # GRPO专用参数
    max_prompt_length=256,
    max_completion_length=1024,
    num_generations=2,
    beta=0.001,
)

trainer = GRPOTrainer(
    model=model_config.model_name_or_path,
    reward_funcs=[format_reward_func, equation_reward_func],
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    peft_config=get_peft_config(model_config),
)

启动训练（注意：强化学习训练速度较慢，在单L4 GPU上，批量大小为1且每组生成2个样本时，单步耗时>20分钟）：

In [ ]:
# 训练并将模型推送至Hub
trainer.train()
# 保存模型
trainer.save_model(training_args.output_dir)

## 4. 使用Deepspeed和vLLM进行GRPO分布式训练示例

单步耗时过长的问题需通过分布式训练解决。Hugging Face TRL支持使用Deepspeed和vLLM加速生成。我已准备[run_r1_grpo.py](https://github.com/philschmid/deep-learning-pytorch-huggingface/blob/main/training/scripts/run_r1_grpo.py)脚本和[grpo-qwen-2.5-3b-deepseek-r1-countdown.yaml](https://github.com/philschmid/deep-learning-pytorch-huggingface/blob/main/training/receipes/grpo-qwen-2.5-3b-deepseek-r1-countdown.yaml)配置文件。

在4×H100 80GB节点上，单步耗时约45-60秒。需注意`num_processes`设为GPU数量减1（最后一个GPU用于vLLM生成）。

运行命令：
```bash
accelerate launch --num_processes 3 --config_file configs/accelerate_configs/deepspeed_zero3.yaml scripts/run_r1_grpo.py --config receipes/grpo-qwen-2.5-3b-deepseek-r1-countdown.yaml
```

优化后，450步训练耗时约6小时。

## 5. 结果与训练观察

训练脚本将生成样本保存至`completion_samples`文件夹。模型检查点可在[philschmid/qwen-2.5-3b-r1-countdown](https://huggingface.co/philschmid/qwen-2.5-3b-r1-countdown)找到。

### 超参数调整
初始使用DeepSeekMath论文中的学习率（1e-6）和beta值（0.04）导致训练不稳定。最终调整为学习率5e-7、beta 0.001。

### 训练观察
- **约50步**：模型学会正确格式`<think>...</think>\n<answer>...</answer>`。
- **100步**：方程解决成功率约25%，开始用自然语言推理。
- **200步**：成功率约40%，推理方式转向程序化试错（见示例）。
- **450步**：成功率50%，性能缓慢提升。

可能原因：
1. Qwen 2.5 3B规模不足。
2. 奖励函数设计偏向方程求解。
3. 训练时长不足（DeepSeek训练了8000+步）。

### 成功推理示例（200步前）
```xml
<think> 需用59、80、66、34得到72。尝试组合：(59 + 66) - 80 = 125 - 80 = 45（错误）→ 最终发现(59 + 66) - 80 = 45。</think>
<answer>(59 + 66) - 80</answer>
```

### 成功推理示例（200-450步）
```xml
<think> 需用75、99、62、34得到72。尝试多种组合后得出：75 + 62 - 99 + 34 = 72。</think>
<answer>75 + 62 - 99 + 34</answer>
```

## 结论

DeepSeek R1的发布标志着开源AI发展的里程碑。我们通过GRPO和《倒计时游戏》复现了其"小顿悟时刻"。尽管实验聚焦特定任务且需大量算力（4×H100训练6小时），但验证了方法的可行性。展望2025年，强化学习将更易用，但也需要更多算力支持。

期待未来更多突破！如有问题或建议，欢迎联系。